In [1]:
import glob
import os
import re

import pandas as pd

r4_l1= pd.read_csv("./raw/sheets/R4コアカリ提出用/第1層.csv").set_index("タブ名")


r4_full= pd.read_csv("./dist/r4_full.csv")
table_df= pd.DataFrame([])
table_files = glob.glob('./raw/sheets/R4コアカリ提出用-別表/別表-*.csv')
for file in table_files:
    df = pd.read_csv(file)
    table = re.search(r'別表\-(.+)\.csv',file).groups()[0]
    df["項目"]=f"[別表]{table}"
    table_df=pd.concat([table_df,df],axis=0)
table_df.to_csv("./dist/r4_table_all.csv",encoding="utf_8_sig",index=False)


r4_counts=r4_full\
    .groupby("第1層",sort=False,as_index=False)\
    .count()\
    .loc[:,["第1層","第4層"]]\
    .rename(columns={"第1層":"項目","第4層":"数"})
r4_counts["項目"]=r4_counts["項目"].str.replace(r"(.+)",r"\1-第4層",regex=True)

table_counts=table_df\
    .groupby(["項目"],sort=False,as_index=False)\
    .count()\
    .loc[:,["項目","H28対応項目"]]\
    .rename(columns={"H28対応項目":"数"})
r4_counts = pd.concat([r4_counts,table_counts],axis=0)

new_line=pd.DataFrame({"項目":[f"合計"],"数":[r4_counts["数"].sum()]})
r4_counts = pd.concat([r4_counts,new_line],axis=0)

r4_counts.to_csv("./dist/r4_counts.csv",encoding="utf_8_sig",index=False)
r4_counts


,項目,数
0,プロフェッショナリズム-第4層,17
1,総合的に患者・生活者をみる姿勢-第4層,83
2,生涯にわたって共に学ぶ姿勢-第4層,19
3,科学的探究-第4層,21
4,専門知識に基づいた問題解決能力-第4層,333
5,情報・科学技術を活かす能力-第4層,13
6,患者ケアのための診療技能-第4層,212
7,コミュニケーション能力-第4層,30
8,多職種連携能力-第4層,21
9,社会における医療の役割の理解-第4層,72


In [2]:
h28=pd.read_csv("./dist/H28.csv")
files = glob.glob('./raw/sheets/*編集用/行き先がないID.csv')
removed_ids= pd.DataFrame([])
for file in files:
    if os.path.getsize(file)<8:
        continue
    df = pd.read_csv(file)
    dir = os.path.basename(os.path.dirname(file))
    tab = re.match(r'(.+?)編集用',dir).group(1)
    df["第1層"]=r4_l1.at[tab,"第1層"]
    df=pd.merge(df,h28.loc[:,["id5","text5"]],left_on="H28ID",right_on="id5",how="left")
    removed_ids = pd.concat([removed_ids,df.loc[:,["第1層","H28ID","理由・コメント","text5"]]],axis=0)

removed_ids=removed_ids.rename(columns={"text5":"H28","理由・コメント":"削除コメント","第1層":"削除担当"})


h28_to_r4 = pd.DataFrame([])
for tab in r4_l1.index:
    df = pd.read_csv(f'./raw/sheets/R4コアカリ提出用/{tab}.csv')
    df["第1層"]=r4_l1.at[tab,"第1層"]
    df["行き先"]=df["第1層"]+"/"+df["第2層"]+"/"+df["第3層"]
    h28_to_r4 = pd.concat([h28_to_r4,df.loc[:,["行き先","H28対応項目"]]],axis=0)


table_df = pd.read_csv('./dist/r4_table_all.csv')
table_h28_to_r4 = table_df.rename(columns={"項目":"行き先"})
h28_to_r4 = pd.concat([h28_to_r4,table_h28_to_r4.loc[:,["行き先","H28対応項目"]]],axis=0)

h28_to_r4= h28_to_r4.rename(columns={"H28対応項目":"H28ID"})


h28_to_r4["H28ID"]=h28_to_r4["H28ID"].str.split(",")
h28_to_r4=h28_to_r4.explode("H28ID")

removed_ids["H28ID"]=removed_ids["H28ID"].str.split(",")
removed_ids=removed_ids.explode("H28ID")

def joinText(split:str):
    def joinNonEmpty(l:list[str]):
        l = list(filter(lambda x: x!="",l))
        return split.join(l) if len(l)>0 else None        
    return joinNonEmpty

h28ids=h28.loc[:,["id5"]].rename(columns={"id5":"H28ID"})
h28ids_to_r4=pd.merge(h28ids,h28_to_r4,on="H28ID",how="outer")
h28ids_to_r4=pd.merge(h28ids_to_r4,removed_ids,on="H28ID",how="outer")
h28ids_to_r4=h28ids_to_r4.fillna("")
h28ids_to_r4=h28ids_to_r4.groupby(["H28ID"],as_index=False,sort=False).agg({
    "行き先":joinText(","),
    "削除担当":joinText(","),
    "削除コメント":joinText(","),
    "H28":joinText(",")
})

removed_ids=pd.merge(removed_ids,h28_to_r4,on="H28ID",how="left")
removed_ids.to_csv("./dist/r4_removed_ids.csv",encoding="utf_8_sig",index=False)
removed_ids
h28_to_r4=pd.merge(h28,h28ids_to_r4,left_on="id5",right_on="H28ID")
h28_to_r4.to_csv("./dist/h28_to_r4.csv",encoding="utf_8_sig",index=False)
h28_to_r4


,index,id1,text1,id2,text2,id3,text3,id4,text4,id5,text5,H28ID,行き先,削除担当,削除コメント,H28
0,1,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-01,医学・医療の歴史的な流れとその意味を概説できる。,A-01-01-na-01,"社会における医療の役割の理解/社会の構造や変化から捉える医療/歴史と医学・医療,社会における...",None,None,None
1,2,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-02,臨床倫理や生と死に関わる倫理的問題を概説できる。,A-01-01-na-02,"社会における医療の役割の理解/社会の構造や変化から捉える医療/哲学・倫理と医療,社会における...",None,None,None
2,3,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-01,医の倫理と生命倫理,A-01-01-na,なし,A-01-01-na-03,ヒポクラテスの誓い、ジュネーブ宣言、医師の職業倫理指針、医師憲章等医療の倫理に関する規範を概...,A-01-01-na-03,社会における医療の役割の理解/社会の構造や変化から捉える医療/哲学・倫理と医療,None,None,None
3,4,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-02,患者中心の視点,A-01-02-na,なし,A-01-02-na-01,リスボン宣言等に示された患者の基本的権利を説明できる。,A-01-02-na-01,"プロフェッショナリズム/思いやり/思いやりと利他,プロフェッショナリズム/思いやり/他者理解...",None,None,None
4,5,A,医師として求められる基本的な資質・能力,A-01,プロフェッショナリズム,A-01-02,患者中心の視点,A-01-02-na,なし,A-01-02-na-02,患者の自己決定権の意義を説明できる。,A-01-02-na-02,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,2002,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-01,シミュレータを用いて反復練習をすることで、臨床技能を磨く。,G-04-04-na-01,None,患者ケアのための診療技能,具体的な目標ではないため,シミュレータを用いて反復練習をすることで、臨床技能を磨く。
2002,2003,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-02,模擬患者の協力を得て、臨床技能（コミュニケーションスキルを含む）や医療者に求められる態度を身...,G-04-04-na-02,"患者ケアのための診療技能/患者の情報収集/医療面接,患者ケアのための診療技能/患者の情報収集...","患者ケアのための診療技能,患者ケアのための診療技能","方略であるため削除し、ガイドラインでの記載を検討(近藤),コミュニケーションへ",模擬患者の協力を得て、臨床技能（コミュニケーションスキルを含む）や医療者に求められる態度を身...
2003,2004,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-03,シナリオを用いたトレーニングを通して、状況判断、意思決定能力を獲得する。,G-04-04-na-03,None,"患者ケアのための診療技能,患者ケアのための診療技能","方略であるため削除し、ガイドラインでの記載を検討(近藤),方略であったため削除(近藤)","シナリオを用いたトレーニングを通して、状況判断、意思決定能力を獲得する。,シナリオを用いたト..."
2004,2005,G,臨床実習,G-04,診療科臨床実習,G-04-04,シミュレーション教育,G-04-04-na,なし,G-04-04-na-04,チームトレーニングによって、チーム医療の実践能力を高める。,G-04-04-na-04,"患者ケアのための診療技能/治療を含む対応の実施/患者ケアに必要な連携,患者ケアのための診療技...","患者ケアのための診療技能,患者ケアのための診療技能","方略であるため削除し、ガイドラインでの記載を検討(近藤),方略であるため","チームトレーニングによって、チーム医療の実践能力を高める。,チームトレーニングによって、チー..."


In [3]:
r4_new = pd.read_csv("./dist/r4_full.csv").loc[:,["第1層","第2層","第3層","第4層","H28対応項目"]]

r4_new= r4_new.rename(columns={"H28対応項目":"H28ID"})
r4_new=r4_new.fillna("")
r4_new=r4_new[r4_new["H28ID"]==""]
r4_new.to_csv("./dist/r4_new.csv",index=False,encoding="utf_8_sig")
r4_new

r4_draft_new = pd.read_csv("./dist/r4_draft.csv").loc[:,["第1層","第2層","第3層","第4層","H28対応項目"]]

r4_draft_new= r4_draft_new.rename(columns={"H28対応項目":"H28ID"})
r4_draft_new=r4_draft_new.fillna("")
r4_draft_new=r4_draft_new[r4_draft_new["H28ID"]==""]
r4_draft_new.to_csv("./dist/r4_draft_new.csv",index=False,encoding="utf_8_sig")
r4_draft_new

,第1層,第2層,第3層,第4層,H28ID
4,プロフェッショナリズム,信頼,省察,他者からのフィードバックを適切に受け入れることができる。,
8,プロフェッショナリズム,思いやり,思いやり,利他を優先できない状況について考えを述べることができる。,
9,プロフェッショナリズム,思いやり,思いやり,削除：自分自身の精神的・身体的健康をどのように管理すればよいか？以下について考え続けそして適...,
12,プロフェッショナリズム,思いやり,品格・礼儀,医師に求められる品格について考え適切に行動できる。,
13,プロフェッショナリズム,思いやり,品格・礼儀,礼儀正しさについて考え適切に行動できる。,
...,...,...,...,...,...
871,,協働実践,関係性への働きかけ,削除：多職種及び他学部の学生との対人葛藤が起きないようにできる。,
873,,協働実践,自職種の省察,削除：自職種が持ちやすい価値観を説明できる。,
875,,協働実践,自職種の省察,自らの価値観や言動を多職種及び他学部の学生間で相対化できる。,
877,,協働実践,他職種の理解,削除：他職種及び他学部の学生の価値観を説明できる。,
